In [1]:
import torch
import gradio as gr
import torch.nn as nn
from sentence_transformers import SentenceTransformer
import datetime

# 1. The Instructions (Model Definition)
class SpearPhishingAE(nn.Module):
    def __init__(self):
        super(SpearPhishingAE, self).__init__()
        self.encoder = nn.Sequential(nn.Linear(384, 128), nn.ReLU(), nn.Linear(128, 64), nn.ReLU(), nn.Linear(64, 32))
        self.decoder = nn.Sequential(nn.Linear(32, 64), nn.ReLU(), nn.Linear(64, 128), nn.ReLU(), nn.Linear(128, 384))
    def forward(self, x):
        return self.decoder(self.encoder(x))

# 2. The Prototype Logic (The "Security Guard" function)
def email_security_pipeline(email_text, model, text_embedder, threshold=0.0012):
    # Convert text to embedding
    embedding = text_embedder.encode([email_text], convert_to_tensor=True).to(torch.device("cpu"))
    
    # AI Scan
    with torch.no_grad():
        reconstruction = model(embedding)
        score = torch.mean((embedding - reconstruction)**2).item()
    
    # Decision logic
    is_threat = score > threshold
    status = "🚩 [BLOCKED]" if is_threat else "✅ [CLEAN]"
    
    return {
        "timestamp": datetime.datetime.now().strftime("%H:%M:%S"),
        "status": status,
        "score": round(score, 6),
        "preview": email_text[:50] + "..."
    }

print("Pipeline Logic Ready! 🛡️")

Pipeline Logic Ready! 🛡️


In [2]:
# Initialize the model and weights first!
device = torch.device("cpu")
model_ae = SpearPhishingAE().to(device)
path = "C:/Users/noaga/OneDrive/Desktop/GitProjects/Cyber_AI_Enhanced_Labs/Project/4.AI Integration and Modeling/models/spear_phishing_ae_weights.pth"
model_ae.load_state_dict(torch.load(path, map_location=device))
model_ae.eval()
text_model = SentenceTransformer('all-MiniLM-L6-v2')

# --- STAGE 8 EVALUATION ---
test_suite = [
    "Meeting reminder: Let's discuss the project budget at 10 AM.", # Safe
    "URGENT: Your account has been compromised. Verify NOW!",   # Threat
]

print(f"{'Time':<10} | {'Status':<12} | {'Score':<10} | {'Message Content'}")
print("-" * 80)

for mail in test_suite:
    result = email_security_pipeline(mail, model_ae, text_model)
    print(f"{result['timestamp']:<10} | {result['status']:<12} | {result['score']:<10} | {result['preview']}")

C:\Users\noaga\AppData\Local\Temp\ipykernel_31216\710143586.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_ae.load_state_dict(torch.load(path, map_location=device

Time       | Status       | Score      | Message Content
--------------------------------------------------------------------------------
09:43:44   | ✅ [CLEAN]    | 0.000983   | Meeting reminder: Let's discuss the project budget...
09:43:44   | 🚩 [BLOCKED]  | 0.001312   | URGENT: Your account has been compromised. Verify ...


In [ ]:
import gradio as gr
import torch
import torch.nn as nn
from sentence_transformers import SentenceTransformer
import datetime

# 1. Logic for the interface
def scan_email_ui(email_text):
    # This uses the same logic we built for the pipeline
    embedding = text_model.encode([email_text], convert_to_tensor=True).to(device)
    with torch.no_grad():
        recon = model_ae(embedding)
        score = torch.mean((embedding - recon)**2).item()
    
    threshold = 0.0015
    if score > threshold:
        result = "🚩 [BLOCKED] - Potential Phishing Threat"
        color = "red"
    elif score > 0.0012:
        result = "⚠️ [Warning] - Suspicious Pattern Detected"
    else:
        result = "✅ [CLEAN] - Safe to Open"
        color = "green"
        
    return result, f"Security Score: {score:.6f}\nTime: {datetime.datetime.now().strftime('%H:%M:%S')}"

# 2. Creating the visual elements
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🛡️ AI Spear Phishing Guard")
    gr.Markdown("Enter an email to check legitimacy")
    
    with gr.Row():
        with gr.Column():
            input_box = gr.Textbox(label="Email Content", placeholder="Paste email here...", lines=5)
            btn = gr.Button("Analyze Message", variant="primary")
        
        with gr.Column():
            output_label = gr.Textbox(label="Security Decision")
            details_box = gr.Textbox(label="System Logs")

    btn.click(fn=scan_email_ui, inputs=input_box, outputs=[output_label, details_box])

# 3. Launch!
demo.launch(debug=True)

C:\Users\noaga\AppData\Local\Temp\ipykernel_31216\4237961281.py:28: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme. Please pass these parameters to launch() instead.
  with gr.Blocks(theme=gr.themes.Soft()) as demo:


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
